In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mibitrans.data.parameter_information import ElectronAcceptors
from mibitrans.data.parameter_information import UtilizationFactor
from mibitrans.data.parameters import AttenuationParameters
from mibitrans.data.parameters import HydrologicalParameters
from mibitrans.data.parameters import ModelParameters
from mibitrans.data.parameters import SourceParameters
from mibitrans.transport.models import Anatrans
from mibitrans.transport.models import Bioscreen
from mibitrans.transport.models import Mibitrans
from mibitrans.visualize.plot_line import breakthrough
from mibitrans.visualize.plot_line import centerline
from mibitrans.visualize.plot_line import transverse
from mibitrans.visualize.plot_surface import plume_2d
from mibitrans.visualize.plot_surface import plume_3d

The showcase of mibitrans below uses example data originating from BIOSCREEN, the Excel based modeling software this package took inspiration from. Describes field conditions on the Keesler Air Force Base in Mississippi, USA. Movement of mixed BTEX plume from 1989 to 1995. For more details, see Newell et al. (1997)

As it was developed in the USA, length units are in ft. We'll use a conversion factor to express length in m instead.

Newell, C. J., McLeod, R. K., & Gonzales, J. R. (1997). BIOSCREEN natural attenuation decision support
system version 1.4 revisions, Tech. rep., U.S. EPA.

In [ ]:
ft = 3.281 # factor to convert ft to m

### Input by dataclasses

mibitrans use dataclasses located in mibitrans.data.read to handle data input. This can be There are five input dataclasses, each for a different category of parameters. To avoid any mistakes, units of input parameters should be the ones specified. When using different units, make sure that they are consistent throughout the entire modelling process.

#### Hydrological parameters

Contains parameters that are inherent to the aquifer properties; flow velocity, porosity and dispersivity. Flow velocity can alternatively be calculated from the hydraulic conductivity and hydraulic gradient. 

In [ ]:
hydro = HydrologicalParameters(
    velocity=113.8/ft/365,   # Groundwater flow velocity, in [m/day]
    porosity=0.25,           # Effective soil porosity [-]
    alpha_x=13.3/ft,         # Longitudinal dispersivity, in [m]
    alpha_y=1.3/ft,          # Transverse horizontal dispersivity, in [m]
    alpha_z=0                # Transverse vertical dispersivity, in [m]
)

# Alternative by specifying hydraulic gradient and hydraulic conductivity
hydro = HydrologicalParameters(
    h_gradient=0.048,        # Hydraulic gradient [-]
    h_conductivity=0.495,    # Hydraulic conductivity [m/day]
    porosity=0.25,           # Effective soil porosity [-]
    alpha_x=13.3/ft,         # Longitudinal dispersivity, in [m]
    alpha_y=1.3/ft,          # Transverse horizontal dispersivity, in [m]
    alpha_z=0                # Transverse vertical dispersivity, in [m]
)

# And then check the value:
print("The calculated groundwater flow velocity is: ", hydro.velocity, r"m/d")

# Any other input parameters can be requested by specifying the argument;
print(f"The dispersivity values are: {hydro.alpha_x}m, {hydro.alpha_y}m and {hydro.alpha_z}m,"
      " for the x, y and z directions respectively.")


#### Attenuation parameters
Handles all parameters related to adsorption, diffusion and degradation.

##### Attenuation: retardation and diffusion
Retardation can be simply given as a value >= 1. Alternatively, the adsorption is calculated from the soil bulk density, paratition coefficient and the fraction of organic carbon in the soil. Note that calculation of retardation factor requires porosity as well, which is already provided in HydrologicalParameters. The calculation of retardation will therefore be automatically performed in the analytical equation. It can be manually calculated using the calculate_retardation method as well.

Here, molecular diffusion can be given as well. It is 0 by default.

In [ ]:
att = AttenuationParameters(
    # Retardation factor for transported contaminants
    retardation=1
)

# Alternatively, calculate the retardation factor by supplying soil and contaminant properties
att = AttenuationParameters(
    # Soil bulk density in [g/m^3]
    bulk_density=1.7,
    # Partition coefficient of the transported contaminant to soil organic matter, in [m^3/g]
    partition_coefficient=38,
    # Fraction of organic material in the soil [-]
    fraction_organic_carbon=5.7e-5
)

# Calculate the retardation factor beforehand to see its value by specifying the porosity
att.calculate_retardation(porosity=hydro.porosity)

# And then check the value:
print("The calculated retardation value is: ", att.retardation)

att = AttenuationParameters(
    # Retardation factor for transported contaminants
    retardation=1,
    # Molecular diffusion, in [m2/day]
    diffusion=1e-5
)

##### Attenuation: degradation parameters

Linear decay models only need either the contaminant decay rate or half life. Input for the instant reaction model is somewhat more involved, and is therefore done with a class method of the model classes.

In [ ]:
att = AttenuationParameters(
    # Contaminant first order decay rate in [1/days]
    decay_rate=0.0127
)

# Alternatively, specify the contaminant half life
att = AttenuationParameters(
    # Contaminant half life, in [days]
    half_life=54.75,
)


For the examples below, we'll set the attenuation parameters to the desired settings

In [ ]:
att = AttenuationParameters(
    # Soil bulk density in [g/m^3]
    bulk_density=1.7,
    # Partition coefficient of the transported contaminant to soil organic matter, in [m^3/g]
    partition_coefficient=38,
    # Fraction of organic material in the soil [-]
    fraction_organic_carbon=5.7e-5,
    # Molecular diffusion, in [m2/day]
    diffusion=0,
    # Contaminant half life, in [days]
    half_life=365#54.75,
)

##### Source parameters

Takes input of the dimensions of and concentrations at the contaminant source. The source is treated as a seperate phase, which dissolves into the groundwater over time. The source is assumed to be in symmetrical in its center, and concentrations decrease from the center to the fringes. Furthermore, the source is assumed to be constant over its depth. The transverse horizontal dimension (or width) of the source is divided into zones, which span a certain distance measured from the center of the source, each with an associated concentration. For example, a source can have a concentration of $10g/m^3$ $7m$ left and right from the source center, and a concentration of $5g/m^3$ up to $20m$ from the source center. This would then be entered as source_zone_boundary = $[7,20]$ and source_zone_concentration = $[10,5]$. The source can be a single zone with a single concentration as well.

By giving a total mass of the contaminant source, the amount of solid-phase contaminant, and with that, the source zone concentrations, diminish over time. The rate at which this occurs depends on the flow velocity in the aquifer and the size of the source zone. The source zone can be set to be considered infinite as well, meaning that the concentrations will not diminish over time.

In [ ]:
# Input for a simple source zone with a width of 19.8m (65ft) and a continuous input (infinite source mass)
source = SourceParameters(
    # Source zone boundaries, in [m] (simply using a float instead of a numpy array for
    # single source zone input will work as well)
    source_zone_boundary=np.array([65/ft]),
    # Source zone concentrations, in [g/m^3]
    source_zone_concentration=np.array([5]),
    # Source depth extent, in [m]
    depth=10/ft,
    # Source mass, considered infinite
    total_mass="inf"
)

# Alternatively, specify a source mass to allow for source decay
source = SourceParameters(
    source_zone_boundary=np.array([7/ft, 37/ft, 65/ft]),
    source_zone_concentration=np.array([13.68, 2.508, 0.057]),
    depth=10/ft,
    total_mass=2000000
)

# Visualize what the source zone looks like to check your input:
source.visualize()
plt.show()

##### Model parameters

Accepts input for the model dimensions and discretization. Model length is the extent in the (x) direction parallel to the groundwater flow direction. The model width is the extent of the model perpendicular (y) to the groundwater flow direction. Step size of the spatial dimensions is handled with dx and dy. Ensure that the source zone fits inside of the given model width. If step sizes are not given, a ratio of model_length (1/100), model_width (1/50) and model_time (1/10) is used by default.

In [ ]:
model = ModelParameters(
    # Model extent in the longitudinal (x) direction in [m].
    model_length = 320/ft,
    # Model extent in the transverse horizontal (y) direction in [m].
    model_width = 100/ft,
    # Model duration in [days].
    model_time = 6 * 365,
    # Model grid discretization step size in the longitudinal (x) direction, in [m].
    dx = 1/ft,
    # Model grid discretization step size in the transverse horizontal (y) direction, in [m].
    dy = 1/ft,
    # Model time discretization step size, in [days]
    dt = 365 / 5
)

##### Input checking

The dataclass inputs evaluates if the required input parameters are present, if they are of the correct data type and if they are in the expected domain.

In [ ]:
# If not all required parameters are specified, an error will be shown;
# HydrologicalParameters needs the porosity, longitudinal dispersivity and transverse horizontal dispersivity as well.
fake_hydro = HydrologicalParameters(velocity = 1)

In [ ]:
# If the input datatype is not what is expected; source_zone_concentration expects a numpy array
# of the same length as the array given in source_zone_boundary.
fake_source = SourceParameters(
    source_zone_boundary = np.array([1,2,3]),
    source_zone_concentration = "this is a string, not an array",
    depth = 10,
    total_mass = "inf",
)

In [ ]:
# Same goes for if the input parameter has a value outside its valid domain; retardation should have a value >= 1
fake_att = AttenuationParameters(
    retardation = 0.1
)

### Analytical models

The various models located in mibitrans.transport.models. Currently, three models are implemented; 'Mibitrans', 'Anatrans' and 'Bioscreen'. Each with a distinct analytical solution, which are introduced below.

Input to the model is provided through the dataclasses showcased above. Each input argument has the same name as the dataclass, but in snake_case instead of PascalCase (e.g. hydrological_paramaters takes in the data class HydrologicalParameters), to follow Python convention. The models are implemented as classes, which are first initialized through making the object. This will generate the model grid and do some prior parameter calculations. The model object can then be used to call attributes, or class methods. After initialization, no results have been calculated yet. For this, use the 'run' class method. This generates a results class object, which contains the concentration distribution as a 3D numpy array (indexed as $C[t,y,x]$) and the parameters used for this specific run.

#### Mibitrans model

The Mibitrans model class uses the exact analytical solution implemented by Karanovic (2007) in the Excel based BIOSCREEN-AT, and added source depletion, akin to that implemented in its predecessor BIOSCREEN by Newell et al. (1997). This model is based on the Wexler (1992) solution. The Mibitrans model allows for the same method as used in BIOSCREEN-AT, but expands it by allowing multiple source zones (by means of superposition) and including the instant reaction model. These were present in the original BIOSCREEN, but not reimplemented in BIOSCREEN-AT. Using a single source zone in this model, and not using the instant reaction option will make the Mibitrans solution resolve to the equation described in Karanovic (2007). Which in turn resolves to the Wexler (1992) solution if source depletion is disabled.

As the namesake model of this package, it is the recommended model to use. The other models introduce a margin of error by making some assumptions. However, since this model requires evaluation of an integral, computation time might be longer, depending on model discretization. The exact nature of these differences is too much to go into detail here, but is elaborated upon in the theoretical background.


In [ ]:
# Initializing the model object
mbt_object = Mibitrans(
    hydrological_parameters=hydro,
    attenuation_parameters=att,
    source_parameters=source,
    model_parameters=model
)

# Can use model object to request various attributes
print("x discretization: ", mbt_object.x)
print("Retarded flow velocity: ", mbt_object.rv)
print("x steps", len(mbt_object.x), "y steps", len(mbt_object.y), "t steps", len(mbt_object.t))

# Run the model once initialized and obtain the results
mbt_results = mbt_object.run()


In [ ]:
?Mibitrans

In [ ]:
# Once the model has run, results are contained in the cxyt attribute
model_cxyt = mbt_object.cxyt

# cxyt is indexed as [time, y-position, x-position]
# Thus to get the concentration at the last time step, in the center of the plume for all x:
plume_center = model_cxyt[-1, 132//2, :]

# mibitrans has build-in visualization methods (see visualization section for more details).
mbt_results.centerline()
plt.show()

To perform the model with biodegradation modelled as an instant reaction, use the instant_reaction class method. Input for the instant reaction model is somewhat more involved, needing electron donor and acceptor concentrations. For utilization factors (amount of electron donor/acceptor used/generated by biodegradation), the values for BTEX degradation are used by default, but custom values can be given. For more specifics about the underlying principles and assumptions, see the theory.

In [ ]:
# For streamlined input, use the ElectronAcceptor dataclass;
ea = ElectronAcceptors(
    # Difference between background oxygen and current oxygen concentration in groundwater, in [g/m^3]
    delta_oxygen=1.65,
    # Difference between background nitrate and current nitrate concentration in groundwater, in [g/m^3]
    delta_nitrate=0.7,
    # Current ferrous iron concentration in groundwater, in [g/m^3]
    ferrous_iron=16.6,
    # Difference between background sulfate and current sulfate concentration in groundwater, in [g/m^3]
    delta_sulfate=22.4,
    # Current methane concentration in groundwater, in [g/m^3]
    methane=6.6
)

mbt_object.instant_reaction(electron_acceptors=ea)

# Can adapt utilization factors if needed
uf = UtilizationFactor(
    # utilization factor of oxygen, as mass of oxygen consumed per mass of biodegraded contaminant [g/g].
    util_oxygen=2,
    # utilization factor of nitrate, as mass of nitrate consumed per mass of biodegraded contaminant [g/g].
    util_nitrate=1,
    # utilization factor of ferrous iron, as mass of ferrous iron generated per mass of biodegraded contaminant [g/g].
    util_ferrous_iron=4,
    # utilization factor of sulfate, as mass of sulfate consumed per mass of biodegraded contaminant [g/g].
    util_sulfate=3,
    # utilization factor of methane, as mass of methane generated per mass of biodegraded contaminant [g/g].
    util_methane=5,
)

mbt_object.instant_reaction(electron_acceptors=ea, utilization_factor=uf)

# Alternatively, electron acceptors and/or utilization factors can be entered as a dictionary
mbt_object.instant_reaction(
    electron_acceptors={
        # Difference between background oxygen and current oxygen concentration in groundwater, in [g/m^3]
        "delta_oxygen":1.65,
        # Difference between background nitrate and current nitrate concentration in groundwater, in [g/m^3]
        "delta_nitrate":0.7,
        # Current ferrous iron concentration in groundwater, in [g/m^3]
        "ferrous_iron":16.6,
        # Difference between background sulfate and current sulfate concentration in groundwater, in [g/m^3]
        "delta_sulfate":22.4,
        # Current methane concentration in groundwater, in [g/m^3]
        "methane":6.6,
    }
)
print(mbt_object.biodegradation_capacity)
# Note that using instant_reaction also resets the utilization factor to default.
print("electron acceptor concentrations: ", mbt_object.electron_acceptors)
print("electron acceptor utilization factors: ", mbt_object.utilization_factor)

Be aware that electron acceptor concentrations and utilization factors can only be changed using the instant_reaction method. Changing the properties directly will not work.

Now that instant reaction parameters are provided, the model can be run:

In [ ]:
mbt_instant_results = mbt_object.run()

mbt_instant_results.centerline()
plt.show()

# Switch between instant reaction and linear model
mbt_object.mode = "linear"
print("The current model mode is: ", mbt_object.mode)
# Running now will show the results of the linear model
# Switch back to instant reaction as follows:
mbt_object.mode = "instant_reaction"
print("The current model mode is: ", mbt_object.mode)

Use the sample method to get the concentration at a specific location and time.

In [ ]:
# Instead, sample a specific location at any point along the plume and any point in time
concentration = mbt_object.sample(
    x_position=100,
    y_position=0,
    time=10*365)
print("The concentration at 150m downstream from the source after 10 years is:", concentration, r"g/m^3")

In [ ]:
# Using the verbose flag, integration steps are printed to console.
# Usefull for longer runs to track progress
mbt_object = Mibitrans(
    hydrological_parameters=hydro,
    attenuation_parameters=att,
    source_parameters=source,
    model_parameters=model,
    verbose=True
)
mbt_results = mbt_object.run()

#### Other models

The two models other than the Mibitrans model share the same functionalities and properties as showcase above. The only difference is the implementation of calculation.

#### Anatrans model
The equation used for the Anatrans model has the assumption that C(x,y,z,t) = C(x,t) * C(y,t) * C(z,t). Then, the 3D ADE can be broken up in three separate differential equations which can be solved individually. For C(x,t) the solution is given in Bear (1979), C(y,t) and C(z,t) can be derived from Crank (1975). The Anatrans model is the combination of these solutions, with addition of source depletion, source superposition and instant reaction model, described in Newell et al. (1997). The solution of Newell et al. (1997) is based of the Domenico (1987) solution, a truncated version of the equation described above, which introduces an error with a size dependent on the ratio of flow velocity and longitudinal dispersivity. Anatrans instead uses the fully untruncated version.

In [ ]:
ana_object = Anatrans(
    hydrological_parameters=hydro,
    attenuation_parameters=att,
    source_parameters=source,
    model_parameters=model
)

ana_results = ana_object.run()
ana_results.centerline()
plt.show()

#### Bioscreen model

This model is an exact implementation of the transport equations implemented in the BIOSCREEN screening model of Newell et al. (1997), which is based on the Domenico (1987) analytical model. Using a truncated version of the equation used in the Anatrans model. This model is implemented as a method of comparison with the original BIOSCREEN software. And is included for legacy reasons, since it is the first model implemented in the mibitrans package, serving as a basis for the other models. However, caution should be taken when using this model, since a varying error is introduced by using the truncated analytical solution. The error is most prominent for shorter times and distances from the source, and depends on the ratio of flow velocity and longitudinal dispersivity. For modelling, the Anatrans (untruncated approximate solution) and Mibitrans (exact analytical solution) models are recommended instead.

In [ ]:
bio_object = Bioscreen(
    hydrological_parameters=hydro,
    attenuation_parameters=att,
    source_parameters=source,
    model_parameters=model
)
bio_results = bio_object.run()
bio_results.centerline()
plt.show()

The model class check if they receive all required and the correct input dataclasses, ensuring model calculations will be performed without error.

In [ ]:
# Will not work; hydrological_parameters should be a HydrologicalParameters class object.
fake_mbt = Mibitrans(
    hydrological_parameters=model,
    attenuation_parameters=att,
    source_parameters=source,
    model_parameters=model
)

## Visualization

Mibitrans has various ways to visualize the model results. Above the centerline plot was already showcased, which plots the concentration distribution over the center of the contaminant plume. Plotting can be performed either as class method (i.e. model_object.centerline), or by calling the centerline function specifically; (i.e. centerline(model_object). The 1D plotting functions are located in mibitrans.visualize.plot_line. Multidimensional plots are located in mibitrans.visualize.plot_surface.

In [ ]:
# Lets make some different model objects to use in the plotting, takes a couple of seconds to run

#Mibitrans model
mbt_object = Mibitrans(hydro, att, source, model)
mbt_lineardecay = mbt_object.run()

mbt_object.attenuation_parameters.decay_rate = 0
mbt_nodecay = mbt_object.run()

mbt_object.instant_reaction(electron_acceptors=ea)
mbt_instant = mbt_object.run()

#Anatrans model
ana_object = Anatrans(hydro, att, source, model)
ana_lineardecay = ana_object.run()

ana_object.attenuation_parameters.decay_rate = 0
ana_nodecay = ana_object.run()

ana_object.instant_reaction(electron_acceptors=ea)
ana_instant =  ana_object.run()

#Bioscreen model
bio_object = Bioscreen(hydro, att, source, model)
bio_lineardecay = bio_object.run()

bio_object.attenuation_parameters.decay_rate = 0
bio_nodecay = bio_object.run()

bio_object.instant_reaction(electron_acceptors=ea)
bio_instant = bio_object.run()



#### Plotting centerline

In [ ]:
# Pass model object to plotting function, by default, the last time step is used.
centerline(mbt_nodecay)
plt.show()

# If you want to plot somewhere and sometime specific, use the time and y_position arguments.
# It gives the concentration profile at the step closest to what you specified.
centerline(mbt_nodecay, time=3*365, y_position=5)
#plt.title("No degradation Domenico model 5m away from center, t=6years")
plt.show()

# If you want you can change the plot settings to the ones you prefer
centerline(mbt_nodecay, time=6*365)
plt.title("Better title than the one generated automatically")
plt.xlabel("I have changed!")
plt.ylabel("And so have I")
plt.show()

legend = ["no degradation", "linear decay", "instant reaction"]

# Instead of a single model, all line visualization functions accept a list of models to be displayed
# together in a single plot.
centerline([mbt_nodecay, mbt_lineardecay, mbt_instant], time=6*365, legend_names=legend)
plt.title("Mibitrans model at plume center for different models, t=6years")
plt.show()

# Keyword arguments for plt.plot can be passed on through the function
# Calling function separately per model gives more control over plot layout
centerline(mbt_nodecay, time=6*365, linestyle="--", color="green", label=legend[0])
centerline(mbt_lineardecay, time=6*365, linestyle="-.", color="red", label=legend[1])
plt.title("No and linear degradation Mibitrans model at plume center, t=6years")
plt.legend()
plt.show()

# Alternatively, use the inherent plotting modules of the model objects
mbt_nodecay.centerline(time = 365, linestyle="--", color="green", label="Mibitrans")
ana_nodecay.centerline(time = 365, linestyle="-.", color="blue", label="Anatrans")
bio_nodecay.centerline(time = 365, linestyle=":", color="red", label="Bioscreen")
plt.title("Concentration distribution for different contaminant transport models after 1 year.")
plt.legend()
plt.show()


#### Plotting transverse distribution

In [ ]:
# Concentration distribution can also be plotted in the transverse direction
transverse(ana_nodecay, x_position=80, time=6*365, linestyle="--", color="green", label="no degradation")
transverse(ana_instant, x_position=80, time=6*365, linestyle="-.", color="red", label="instant reaction")

plt.title("No degradation and instant reaction Domenico model at x=80m, t=6years")
plt.legend()
plt.show()

# Or using the class methods
ana_lineardecay.transverse(x_position=80, time=6*365)
plt.show()

#### Plotting breakthrough curve

In [ ]:
# Concentration distribution can also be plotted in the transverse direction
breakthrough([bio_nodecay, bio_lineardecay, bio_instant], x_position=80,
             legend_names=["no degradation", "linear decay", "instant reaction"])
plt.title("Breakthrough curve of Bioscreen model for different degradation settings, at x=80m, t=6years")
plt.show()


#### Plotting in 2D

In [ ]:
# Plot the x and y concentration distribution for the Mibitrans model, uses plt.pcolormesh
plume_2d(mbt_nodecay, time=6*365)
plt.title("Contaminant plume with no degradation Mibitrans model, t = 6 years")
plt.show()

# Function passes plt.colormesh keyword arguments
plume_2d(mbt_lineardecay, time=6*365, cmap="magma")
plt.title("Contaminant plume with linear degradation Domenico model, t = 6 years")
plt.show()

# Once again also can be accessed through the class method
mbt_instant.plume_2d(time=4*365, cmap="plasma")
plt.show()

#### Plotting 2D in 3D

In [ ]:
# Plot the x and y concentration distribution for no degradation decay model, uses plot_surface
plume_3d(ana_nodecay, time=6*365)
plt.title("Contaminant plume with no degradation Anatrans model, t = 6 years")
plt.show()

# Function passes plot_surface keyword arguments
plume_3d(ana_lineardecay, time=6*365, cmap="viridis")
plt.title("Contaminant plume with linear degradation Anatrans model, t = 6 years")
plt.show()

# Function returns 'ax' object, use this to change view point of plot
# And can be accessed through class methods
ax = ana_instant.plume_3d(time=6*365, cmap="viridis")
plt.title("Contaminant plume with linear degradation Anatrans model, t = 6 years")
ax.view_init(elev=15, azim=340)
plt.show()

#### Animate plots

All plots mentioned above in the visualization section have the option to be animated, which also can be saved as a file. Multiple models can be combined in a single animation. Make sure that parameters passed to the functions are inside the domain of all models. As each animation frame is a model time step, all models should have the exact same dt, otherwise, the animation will not show the correct temporal change in concentration.

In [ ]:
# Needed to show animations in Jupyter Notebooks
%matplotlib notebook

In [ ]:
# Output needs to be assigned to variable for animation to work
ani = centerline([mbt_nodecay, mbt_lineardecay, mbt_instant], time=6*365, legend_names=legend, animate=True)
plt.show()

# Animation of breakthrough curve instead shows a timelapse of drawing of each curve
ani1 = breakthrough([bio_nodecay, bio_lineardecay, bio_instant], x_position=20, legend_names=legend, animate=True)
plt.show()

# For the 3d surface plot, an entirely new plot needs to be generated per time step
# This can cause slightly longer execution times
ani2 = ana_instant.plume_3d(time=6*365, animate=True, cmap="viridis")
plt.show()

In [ ]:
# Save animation as .gif (or other desired file format), this may take a while
# Adjust animation speed with optional fps argument
ani2.save("plume_animation.gif", fps=10)

## Mass balance

To gain numerical information about mass transport in the model area, use the mass balance method of the Results object. Each of the mass balance elements is a class property. Descriptions of each mass balance element can be found in analysis.mass_balance.MassBalance.

In [ ]:
# Set up a mass balance object of the Mibitrans instant reaction model.
mb = mbt_instant.mass_balance()

As the warning shows, part of the plume extents beyond the model boundary, which makes the mass balance inaccurate. Increasing model length will remediate this.

In [ ]:
mbt_object.model_parameters.model_length = 500 / ft
results = mbt_object.run()
mb = results.mass_balance()

The entire plume is now inside the model extent.

In [ ]:
results.centerline()
plt.show()
results.transverse(20, label="x = 20m")
results.transverse(60, label="x = 60m")
results.transverse(100, label="x = 100m")
results.transverse(140, label="x = 140m")
plt.legend()
plt.title("Transverse plot of Mibitrans instant reaction model, at t=6 years")
plt.show()

In [ ]:
# By default, mass balance gives results for each model time step
print(mb.plume_mass)
# And then can easily be plotted over time
plt.plot(mb.t, mb.plume_mass)
plt.xlabel("time (days)")
plt.ylabel("plume mass (g)")
plt.show()
plt.plot(mb.t, mb.degraded_mass)
plt.xlabel("time (days)")
plt.ylabel("degraded mass (g)")
plt.title("Degraded plume mass of instant reaction model, compared to no decay.")
plt.show()